In [ ]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 5000
N_bound = 100

np.random.seed(123)
train_x_y_t = lhs(3,N_train)
np.random.seed(12345)
bound_x_y = lhs(2,N_bound)
bound_x_y_t = np.concatenate((bound_x_y,np.zeros((N_bound,1))),1)

In [ ]:
train_x_y_t = torch.from_numpy(train_x_y_t).float()
bound_x_y = torch.from_numpy(bound_x_y).float()
bound_x_y_t = torch.from_numpy(bound_x_y_t).float()

x = train_x_y_t[:,0:1]
y = train_x_y_t[:,1:2]
t = train_x_y_t[:,2:3]
x.requires_grad_()
y.requires_grad_()
t.requires_grad_()

In [ ]:
def relative_l2(u_pred,u_real):
    l2 = np.linalg.norm(u_real-u_pred,2)/np.linalg.norm(u_real,2)
    return l2

In [ ]:
import scipy.integrate
from numpy import exp
from math import sqrt
import math
def InIn(x_y_t):
    x1 = x_y_t[:,0:1]
    x2 = x_y_t[:,1:2]
    t = x_y_t[:,2:3]
    P = np.zeros_like(t)
    for i in range(len(t)):
        f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t[i]))*(y1-y2)
        p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
        P[i] = p/(4*np.pi*t[i])
    return P

In [ ]:
np.random.seed(5678)
test_x_y_t = lhs(3,500)
x1 = test_x_y_t[:,0:1]
x2 = test_x_y_t[:,1:2]
t_ = test_x_y_t[:,2:3]
test_u = np.zeros_like(t_)
for i in range(len(t_)):
    f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t_[i]))*(y1-y2)
    p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
    test_u[i] = p/(4*np.pi*t_[i])


In [ ]:
test_x_y_t = torch.from_numpy(test_x_y_t).float()

### Labeled Data

In [ ]:
np.random.seed(234)
gan_data_x_t = lhs(3,10)
gan_data_u = InIn(gan_data_x_t)

In [ ]:
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.from_numpy(gan_data_u).float()

# Method1
$\text { PINN }$

<div class="alert alert-info">`loss_function：`
$\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}$</div>

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs1 = NN_H2(3, 100, 4, 1)
PINNs1.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs1.parameters()}], lr=1e-3)



loss_all_1 = []
test_loss_1 = []

#########gpu############
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs1.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############

loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs1.cuda()
    
    ##### loss_Bi  ######
    u_bound = PINNs1(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs1(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    loss = loss_bound+loss_f
    #loss = loss_f 
    loss1_value = loss.item()
    
    
    loss_all_1.append(loss1_value)
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    
    #########  test_loss NRMSE  #########
    PINNs1.cpu()
    test_loss =  relative_l2(PINNs1(test_x_y_t).detach().numpy(),test_u)
    test_loss_1.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

# Results

## Save data

In [ ]:
# np.save('../experimental_data/method_1/test_loss_1',test_loss_1)
# np.save('../experimental_data/method_1/loss_all_1',loss_all_1)
# torch.save(PINNs1,'../saved_model/PINNs1')

## Epoch and NRMSE

In [ ]:
Epochs = len(test_loss_1)
NRMSE = relative_l2(PINNs1(test_x_y_t).detach().numpy(),test_u)

print('Epochs:',Epochs,'NRMSE:',NRMSE)